# Rounding Schemes
This notebook explores rounding schemes (and compares them against numpy.round()).

From comments in [numpy.round()](https://github.com/numpy/numpy/blob/e94ed84010c60961f82860d146681d3fd607de4e/numpy/core/fromnumeric.py#L2781), it appears that it rounds to the nearest even number:

    >>> np.around([.5, 1.5, 2.5, 3.5, 4.5]) # rounds to nearest even value
    array([ 0.,  2.,  2.,  4.,  4.])
    
This is called [rounding half to even](https://en.wikipedia.org/wiki/Rounding#Rounding_half_to_even) or banker's rounding (see [this](https://stackoverflow.com/a/10825998)).

In this notebook, we compare the bias in different rounding schemes (as suggested [here](https://stackoverflow.com/a/23385667)).

## Stochastic Rounding
The idea here is to round x.5 up or down based on a coin flip:

In [3]:
from math import modf
import numpy as np

np.random.seed = 2021.698635


def stochastic_round(num):
    # get the integer and fractional parts of num:
    num_frac, num_int = modf(num)

    # now round up, or down, or do a coin flip first
    if num_frac > 0.5:
        num_int += 1  # round up
    elif num_frac < 0.5:
        num_int -= 1  # round down
    else:  # num_frac == 0.5
        coin_flip = np.random.rand() > 0.5  # or should we use >= ?
        if coin_flip:
            num_int += 1
        else:
            num_int -= 1

    return num_int

Now we test this idea out, comparing the rounding function above against numpy:

In [5]:
# List of numbers to try rounding
test_cases = [0.5, 1.5]

# Number of repetitions to run
num_iter = 10000

for test_num in test_cases:
    # tracking rounding results
    stoc_up_cnt = 0
    np_up_cnt = 0

    # loop test!
    for n in range(num_iter):
        stoc_rounded = stochastic_round(test_num)
        if stoc_rounded > test_num:
            stoc_up_cnt += 1

        np_rounded = np.round(test_num)
        if np_rounded > test_num:
            np_up_cnt += 1

    print(
        "Rounding {}: numpy rounds up\t{} of {} times\t[{:.2f}%]".format(
            test_num, np_up_cnt, num_iter, 100 * np_up_cnt / num_iter
        )
    )
    print(
        "Rounding {}: stoch rounds up\t{} of {} times\t[{:.2f}%]".format(
            test_num, stoc_up_cnt, num_iter, 100 * stoc_up_cnt / num_iter
        )
    )

Rounding 0.5: numpy rounds up	0 of 10000 times	[0.00%]
Rounding 0.5: stoch rounds up	5055 of 10000 times	[50.55%]
Rounding 1.5: numpy rounds up	10000 of 10000 times	[100.00%]
Rounding 1.5: stoch rounds up	4972 of 10000 times	[49.72%]
